In [49]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.iv import IV2SLS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Sales,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,const
8,1,2021,257,0.010319,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
9,1,2022,183,0.005938,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
10,1,2023,177,0.002822,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
21,2,2023,3,0.000048,Aiways,U6,405,3.757100e+05,214,34,SUV,C,CH,1.0
28,3,2019,222,0.040217,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
29,3,2020,491,0.034512,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
30,3,2021,260,0.010440,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
31,3,2022,538,0.017456,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
32,3,2023,68,0.001084,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
41,4,2021,84,0.003373,Audi,e-tron GT,472,1.081416e+06,522,17,Sedan,F,DE,1.0


In [51]:
# Copy the dataframe
df2 = data.copy()

In [52]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [53]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

In [54]:
y = df2['log_market_share']
x = df2[['const']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['HP', 'Range', 'Chargetime']]

In [55]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='unadjusted')

In [56]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                      0.1371
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0871
No. Observations:                 348   F-statistic:                    176.17
Date:                Tue, Mar 12 2024   P-value (F-stat)                0.0000
Time:                        22:04:26   Distribution:                 chi2(19)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.7217     0.9976    -5.7355     0.0000     -7.6769     -3.7664
Segment_B     -0.0167     0.6006    -0.0277     0.9779     -1.1938      1.1605
Segment_C      0.1903     0.7049     0.2699     0.7872     -1.1913      1.5719
Segment_D     -0.9425     1.1062    -0.8520     0.3942     -3.1107      1.2257
Segment_E     -4.2538     1.5575    -2.7311     0.0063     -7.3064     -1.2011
Segment_F     -7.1529     2.4796    -2.8847     0.0039     -12.013     -2.2930
Segment_M     -3.6928     1.0366    -3.5623     0.0004     -5.7245     -1.6610
Segment_J     -0.6388     2.3585    -0.2709     0.7865     -5.2615      3.9838
Year_2014      0.5422     1.1608     0.4671     0.6404     -1.7328      2.8173
Year_2015     -0.0138     1.1280    -0.0122     0.9902     -2.2247      2.1970
Year_2016      0.0129     1.0977     0.0117     0.9906     -2.1385      2.1643
Year_2017      0.0595     1.0756     0.0553     0.9559     -2.0486      2.1676
Year_2018     -0.0245     1.0566    -0.0232     0.9815     -2.0955      2.0465
Year_2019     -0.4902     0.9896    -0.4953     0.6204     -2.4298      1.4495
Year_2020     -1.4099     0.9037    -1.5602     0.1187     -3.1812      0.3613
Year_2021     -2.2850     0.8747    -2.6123     0.0090     -3.9994     -0.5706
Year_2022     -2.5910     0.8663    -2.9910     0.0028     -4.2888     -0.8932
Year_2023     -3.1858     0.8639    -3.6877     0.0002     -4.8789     -1.4926
China         -0.0141     0.5094    -0.0277     0.9779     -1.0126      0.9844
Price       8.801e-06  2.739e-06     3.2135     0.0013   3.433e-06   1.417e-05
==============================================================================

Endogenous: Price
Instruments: HP, Range, Chargetime
Unadjusted Covariance (Homoskedastic)
Debiased: False
"""

# Test

In [57]:
df1 = pd.read_csv('dataset.csv')
Nobs=df1['ID'].count()
df1['const']=np.ones((Nobs,1))
df1 = df1[df1['Market share'] != 0]
#df1['HP_instrument'] = 0
df1.head(20)

,ID,Year,Sales,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,const
8,1,2021,257,0.010319,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
9,1,2022,183,0.005938,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
10,1,2023,177,0.002822,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
21,2,2023,3,0.000048,Aiways,U6,405,3.757100e+05,214,34,SUV,C,CH,1.0
28,3,2019,222,0.040217,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
29,3,2020,491,0.034512,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
30,3,2021,260,0.010440,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
31,3,2022,538,0.017456,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
32,3,2023,68,0.001084,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
41,4,2021,84,0.003373,Audi,e-tron GT,472,1.081416e+06,522,17,Sedan,F,DE,1.0


In [58]:
models = df1['Model'].unique()
HP = df1['HP']
print(models)

['U5' 'U6' 'e-tron' 'e-tron GT' 'Q4 e-tron' 'Q8 e-tron' 'I3' 'i4' 'i5'
 'I7' 'iX' 'iX1' 'iX3' 'Atto 3' 'Dolphin' 'Han' 'Seal' 'Tang' 'Berlingo'
 'C4' 'ë-C4 X' 'e-SpaceTourer' 'Jumpy' 'Born' 'Spring' '3 Crossback' '500'
 'Ocean' 'Mustang Mach-E' 'E' 'e:Ny1' 'E-HS9' 'Ioniq' 'Ioniq 5' 'Ioniq 6'
 'Kona' 'e-JS4' 'I-Pace' 'Avenger' 'EV6' 'EV9' 'Niro' 'Soul' 'RZ' 'UX'
 'Euniq' 'Euniq5' 'Euniq6' 'MIFA 9' 'Mifa9' 'MX-30' 'B-Klasse' 'EQA' 'EQB'
 'EQC' 'EQE' 'EQE SUV' 'EQS' 'EQS SUV' 'EQV' 'Vito' 'Vito Tourer' '4' '5'
 'Marvel R' 'ZS' '3-dørs' 'Cabriolet' 'EL6' 'EL7' 'ES8' 'ET5' 'ET7'
 'Ariya' 'Leaf' 'Combo' 'Corsa' 'Mokka' 'Zafira' '2008' '208' 'Expert'
 'Rifter' 'Traveller' '2' 'Taycan' 'Megane' 'Zoe' 'Mii' 'Citigo'
 'Enyaq iV' 'Fortwo' 'Korando' 'Solterra' 'Model 3' 'Model S' 'Model X'
 'Model Y' 'bZ4X' 'Proace City Verso' 'Proace Verso' 'Golf' 'ID.3' 'ID.4'
 'ID.5' 'ID.7' 'ID.Buzz' 'up!' 'C40' 'EX30' 'XC40' 'Free']


In [59]:
def HP_instrument1(df1):
    z = 0
    models = df1['Model'].unique()
    for model in models:
        for i in df1:
            if df1[model] == df1['Model'][i]:
                z = z
            else:
                z += df1['HP'][i]
    df1['HP_instrument'] = z            

In [60]:
def create_instrument(df1, instrument):
    df1[instrument+'_instrument'] = 0
    for index, row in df1.iterrows():
        current_model = row['Model']
        hp_sum_except_current = df1[df1['Model'] != current_model][instrument].sum()
        df1.at[index, instrument+'_instrument'] = hp_sum_except_current


In [61]:
create_instrument(df1, 'Chargetime')

In [62]:
df1

,ID,Year,Sales,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,const,Chargetime_instrument
8,1,2021,257,0.010319,Aiways,U5,400,330825.789474,201,34,SUV,C,CH,1.0,12087
9,1,2022,183,0.005938,Aiways,U5,400,330825.789474,201,34,SUV,C,CH,1.0,12087
10,1,2023,177,0.002822,Aiways,U5,400,330825.789474,201,34,SUV,C,CH,1.0,12087
21,2,2023,3,0.000048,Aiways,U6,405,375710.000000,214,34,SUV,C,CH,1.0,12155
28,3,2019,222,0.040217,Audi,e-tron,375,714953.030167,402,17,SUV,F,DE,1.0,12104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,189,2023,4,0.000064,Volvo,EX30,475,368245.000000,268,28,SUV,B,SE,1.0,12161
1218,190,2021,363,0.014575,Volvo,XC40,457,443449.849520,402,28,SUV,C,SE,1.0,12105
1219,190,2022,1020,0.033094,Volvo,XC40,457,443449.849520,402,28,SUV,C,SE,1.0,12105
1220,190,2023,1965,0.031327,Volvo,XC40,457,443449.849520,402,28,SUV,C,SE,1.0,12105
